In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 22.6 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19580 sha256=a9f5e6d38bcf1188d66bf8109172d66030130ca015a97f69cb23c762f78880c9
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 5.0.1
    Uninstalling tzloca

In [ ]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 31.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.7 requires jax>=0.4.6, but you have jax 0.3.25 which is incompatible.
flax 0.7.0 requires jax>=0.4.2, but you have jax 0.3.25 which is incompatible.
orbax-checkpoint 0.3.1 requires jax>=0.4.9, but you have jax 0.3.25 which is incompatible.


In [ ]:
import tensorflow as tf
import streamlit as st
import numpy as np

In [ ]:
def load_model():
    model_path = "/content/drive/MyDrive/Model_GUI1/H2S_Model_GUI.h5"
    model = tf.keras.models.load_model(model_path)
    return model

In [ ]:
model = load_model()

In [ ]:
model

In [ ]:
%%writefile H2S.py
import streamlit as st
import numpy as np
import tensorflow as tf
import base64

# Custom function to load the model
def load_model():
    model_path = "/content/drive/MyDrive/Model_GUI1/H2S_Model_GUI.h5"
    model = tf.keras.models.load_model(model_path)
    return model

model = load_model()

# Custom function for preprocessing input data
def preprocess(mea_gas, air_mea1, air_mea2, air_sws, sws_gas):
    num_features = 5
    input_data = np.array([[mea_gas, air_mea1, air_mea2, air_sws, sws_gas]])
    input_data = input_data.reshape((1, 1, num_features))
    return input_data

# Custom function for LSTM inference
def run_lstm_inference(mea_gas, air_mea1, air_mea2, air_sws, sws_gas):
    input_data = preprocess(mea_gas, air_mea1, air_mea2, air_sws, sws_gas)
    predictions = model.predict(input_data)
    return predictions

def main():
    # Custom UI design using Streamlit
    st.set_page_config(
        page_title="LSTM Soft Sensor for SRU Measurement",
        page_icon="🧪",
        layout="wide",
    )

    st.title("LSTM Soft Sensor for SRU Measurement")

    # Convert image file to base64 string
    background_image_path = "/content/drive/MyDrive/fotor-ai-20230801102056.jpg"
    with open(background_image_path, "rb") as img_file:
        base64_image = base64.b64encode(img_file.read()).decode()

    # Use base64_image in the background-image property
    st.markdown(
        f"""
        <style>
        body {{
            background-image: url('data:image/png;base64,{base64_image}');
            background-size: cover;
            background-repeat: no-repeat;
            background-attachment: fixed;
            background-position: center;
        }}
        </style>
        """,
        unsafe_allow_html=True
    )

    st.markdown(
        """
        This is a simple LSTM-based soft sensor to predict SRU flow rate measurements.
        Please enter the values for the input features and click the **Predict** button to get the predictions.
        """
    )

    mea_gas = st.number_input("MEA GAS", value=0.0, step=0.001, format="%.6f")
    air_mea1 = st.number_input("AIR MEA1", value=0.0, step=0.001, format="%.6f")
    air_mea2 = st.number_input("AIR MEA2", value=0.0, step=0.001, format="%.6f")
    air_sws = st.number_input("AIR SWS", value=0.0, step=0.001, format="%.6f")
    sws_gas = st.number_input("SWS GAS", value=0.0, step=0.001, format="%.6f")

    if st.button("Predict"):
        predictions = run_lstm_inference(mea_gas, air_mea1, air_mea2, air_sws, sws_gas)

        # Show input values
        st.subheader("Input Values")
        st.write("MEA GAS:", mea_gas)
        st.write("AIR MEA1:", air_mea1)
        st.write("AIR MEA2:", air_mea2)
        st.write("AIR SWS:", air_sws)
        st.write("SWS GAS:", sws_gas)

        # Show predicted values
        st.subheader("Predicted Values")
        st.write("Predictions:", predictions)

        # Interpretation
        st.subheader("Interpretation")
        st.write("The LSTM model predicts the following SRU flow rate values based on the input data.")

if __name__ == "__main__":
    main()


Writing H2S.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.123.175.244


In [ ]:
!streamlit run H2S.py & npx localtunnel --port 8501

[..................] - fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.193.188.233:8501

npx: installed 22 in 3.396s
your url is: https://legal-stars-help.loca.lt
2023-08-01 10:23:50.259774: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
1/1 [==============================] - 2s 2s/step
